# LinearModel

In [ ]:
task = "random"

In [ ]:
import SparseArrays: sparse
import NBInclude: @nbinclude
@nbinclude("../Alpha.ipynb")
@nbinclude("EnsembleInputs.ipynb");

## Suppress seen shows
* Zero out any items the user has already seen and rescale the probablity distribution

In [ ]:
function get_seen_probabilities(alpha::String)
    if alpha in implicit_raw_alphas(task)
        content = "implicit"
    elseif alpha in ptw_raw_alphas(task)
        content = "ptw"
    else
        @assert false
    end
    df = read_alpha(alpha, "training", task, content)
    p_seen = zeros(Float32, num_users(), Threads.nthreads())
    Threads.@threads for i = 1:length(df.user)
        p_seen[df.user[i], Threads.threadid()] += df.rating[i]
    end
    vec(sum(p_seen, dims = 2))
end

function read_raw_alpha(alpha::String, split::String, task::String, content::String)
    suppress =
        (split != "training") &&
        (alpha in [implicit_raw_alphas(task); ptw_raw_alphas(task)])
    if !suppress
        return read_raw_alpha_impl(alpha, split, task, content)
    end
    ϵ = sqrt(eps(Float32))
    df = get_raw_split(split, task, content; fields = [:user])
    ratings = read_raw_alpha_impl(alpha, split, task, content).rating
    # zero out any items the user has already seen
    p_seen = get_seen_probabilities(alpha)
    Threads.@threads for i = 1:length(ratings)
        if 1 - p_seen[df.user[i]] > ϵ
            ratings[i] /= 1 - p_seen[df.user[i]]
        end
    end
    RatingsDataset(rating = ratings)
end;

## Save predictions

In [ ]:
function save_linear_model(
    alphas::Vector{String},
    task::String,
    content::String,
    outdir::String,
)
    if content == "explicit"
        implicit = false
    elseif content in ["implicit", "ptw"]
        implicit = true
    else
        @assert false
    end
    set_logging_outdir(outdir)
    _, β = regress(alphas, task, content, implicit)
    @info "alphas: $alphas"
    @info "coefficients: $β"
    write_params(Dict("β" => β, "alphas" => alphas), outdir)

    function model(split::String, task::String, content::String; raw_splits::Bool)
        read_fn = raw_splits ? read_raw_alpha : read_alpha
        Xs = [read_fn(alpha, split, task, content).rating for alpha in alphas]
        if implicit
            push!(Xs, fill(1.0f0 / num_items(), length(Xs[1])))
        end
        X = hcat(Xs...)
        y = X * β
        vec(y)
    end
    write_alpha(
        model,
        outdir;
        task = task,
        by_split = true,
        log = true,
        log_alphas = String[],
        log_task = task,
        log_content = content,
        log_splits = ["validation", "test"],
    )
end;

In [ ]:
save_linear_model(explicit_raw_alphas(task), task, "explicit", "$task/LinearExplicit");

In [ ]:
save_linear_model(implicit_raw_alphas(task), task, "implicit", "$task/LinearImplicit");

In [ ]:
save_linear_model(ptw_raw_alphas(task), task, "ptw", "$task/LinearPtw");